<a href="https://colab.research.google.com/github/SherifNabil-Eng/AzureCognitivePoCs/blob/main/TestAzureSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install azure-ai-textanalytics==5.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 636.7 kB/s eta 0:00:00


In [8]:
! pip install python-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 34.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184505 sha256=43487b55ab0f44bd11798cecd1372c5c9628cbb7828f993bce972290d95d4f43
  Stored in directory: /root/.cache/pip/wheels/32/b8/b2/c4c2b95765e615fe139b0b17b5ea7c0e1b6519b0a9ec8fb34d
Successfully built python-docx


In [9]:
# import libraries
from google.colab import drive
drive.mount('/content/gdrive')

import os
import numpy as np
import pandas as pd
from datetime import date
import sys
#reload(sys)


import os
import locale
#os.environ["PYTHONIOENCODING"] = "utf-8"
import docx


from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError


os.chdir("/content/gdrive/My Drive/Colab Notebooks/Tests")
#sys.setdefaultencoding('utf-8')
#!locale-gen en_US.UTF-8
#!export LANG=en_US.UTF-8 LANGUAGE=en_US.en
#!LC_ALL=en_US.UTF-8

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
language_endpoint = "https://cognservgen.cognitiveservices.azure.com/"
language_key = "fba635423d6547248dd57386dd6220e4"

# Authenticate the client using your key and endpoint 
def authenticate_client():
    ta_credential = AzureKeyCredential(language_key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=language_endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

In [16]:
# Example method for detecting sentiment and opinions in text 
def sentiment_analysis_with_opinion_mining_example(client,documents,key_phrases):

    #documents = [
    #    "The food and service were unacceptable. The concierge was nice, however."
    #]

    result = client.analyze_sentiment(documents, show_opinion_mining=True)
    doc_result = [doc for doc in result if not doc.is_error]

    positive_reviews = [doc for doc in doc_result if doc.sentiment == "positive"]
    negative_reviews = [doc for doc in doc_result if doc.sentiment == "negative"]

    positive_mined_opinions = []
    mixed_mined_opinions = []
    negative_mined_opinions = []

    for document in doc_result:
        print("Document Sentiment: {}".format(document.sentiment))
        print("Overall scores: positive={0:.2f}; neutral={1:.2f}; negative={2:.2f} \n".format(
            document.confidence_scores.positive,
            document.confidence_scores.neutral,
            document.confidence_scores.negative,
        ))
        
        key_phrases.append("\n Document Sentiment: {}".format(document.sentiment))
        key_phrases.append ("\n Overall scores: positive={0:.2f}; neutral={1:.2f}; negative={2:.2f} \n".format(
            document.confidence_scores.positive,
            document.confidence_scores.neutral,
            document.confidence_scores.negative,
        ))

        for sentence in document.sentences:
            print("Sentence: {}".format(sentence.text))
            print("Sentence sentiment: {}".format(sentence.sentiment))
            print("Sentence score:\nPositive={0:.2f}\nNeutral={1:.2f}\nNegative={2:.2f}\n".format(
                sentence.confidence_scores.positive,
                sentence.confidence_scores.neutral,
                sentence.confidence_scores.negative,
            ))

            key_phrases.append ("\n Sentence: {}".format(sentence.text))
            key_phrases.append ("\n Sentence sentiment: {}".format(sentence.sentiment))
            key_phrases.append ("Sentence score:\nPositive={0:.2f}\nNeutral={1:.2f}\nNegative={2:.2f}\n".format(
                sentence.confidence_scores.positive,
                sentence.confidence_scores.neutral,
                sentence.confidence_scores.negative,
            ))

            for mined_opinion in sentence.mined_opinions:
                target = mined_opinion.target
                print("......'{}' target '{}'".format(target.sentiment, target.text))
                print("......Target score:\n......Positive={0:.2f}\n......Negative={1:.2f}\n".format(
                    target.confidence_scores.positive,
                    target.confidence_scores.negative,
                ))
                for assessment in mined_opinion.assessments:
                    print("......'{}' assessment '{}'".format(assessment.sentiment, assessment.text))
                    print("......Assessment score:\n......Positive={0:.2f}\n......Negative={1:.2f}\n".format(
                        assessment.confidence_scores.positive,
                        assessment.confidence_scores.negative,
                    ))
            print("\n")
        print("\n")
          


In [6]:
#sentiment_analysis_with_opinion_mining_example(client)

Document Sentiment: mixed
Overall scores: positive=0.49; neutral=0.00; negative=0.50 

Sentence: The food and service were unacceptable. 
Sentence sentiment: negative
Sentence score:
Positive=0.00
Neutral=0.00
Negative=1.00

......'negative' target 'food'
......Target score:
......Positive=0.01
......Negative=0.99

......'negative' assessment 'unacceptable'
......Assessment score:
......Positive=0.01
......Negative=0.99

......'negative' target 'service'
......Target score:
......Positive=0.01
......Negative=0.99

......'negative' assessment 'unacceptable'
......Assessment score:
......Positive=0.01
......Negative=0.99



Sentence: The concierge was nice, however.
Sentence sentiment: positive
Sentence score:
Positive=0.98
Neutral=0.01
Negative=0.01

......'positive' target 'concierge'
......Target score:
......Positive=1.00
......Negative=0.00

......'positive' assessment 'nice'
......Assessment score:
......Positive=1.00
......Negative=0.00







In [11]:
# Read the Document file

def read_file (file_sub):

  curren_root=os.getcwd()
  file_Path =  os.path.join(curren_root,file_sub)
    
 
  # open connection to Word Document
  doc_temp = docx.Document(file_Path)
 
  # read in each paragraph in file
  document = [p.text for p in doc_temp.paragraphs]
 

  #remove('')
  document_trimmed  = [sentence for sentence in document if (sentence != '\n')]
  document_trimmed  = [sentence for sentence in document if (sentence != '')]
  #print (document_trimmed[0:30])
  #print ("\n length of the document trimmed is \n",len(document_trimmed))
  #print ("document type is ",type (document_trimmed))
  return document_trimmed

In [17]:
if __name__ == "__main__":
  file_sub="TextExtractDocs/TranscriptFY22Q4.docx"
  key_phrases = []

# enhance i as it is related to the current document
  document = read_file(file_sub)
  #print ("Length of document is \n ",document)
  for i in range (10,len (document),10):
    sentiment_analysis_with_opinion_mining_example(client,document[i-10:i],key_phrases)
    #sample_extractive_summarization(document[i-10:i],i-10,key_phrases)


Streaming output truncated to the last 5000 lines.

Sentence: Over the past year, we’ve introduced more than 450 capabilities to empower frontline and knowledge workers to collaborate both synchronously and asynchronously, as well as remote and in person. 
Sentence sentiment: neutral
Sentence score:
Positive=0.01
Neutral=0.99
Negative=0.00





Document Sentiment: neutral
Overall scores: positive=0.01; neutral=0.98; negative=0.00 

Sentence: Teams is taking share across every category, from collaboration, to chat, to meetings, to calling – and seeing higher usage intensity as companies turn to the platform to accelerate their digital transformation and orchestrate all their business processes in the flow of work. 
Sentence sentiment: neutral
Sentence score:
Positive=0.01
Neutral=0.98
Negative=0.00





Document Sentiment: neutral
Overall scores: positive=0.00; neutral=0.99; negative=0.00 

Sentence: ISVs, from Adobe to Workday, have built deep integrations with Teams. 
Sentence sentime

In [18]:
index_slash=file_sub.find ("/")
index_dot=file_sub.find (".")
file_sub_name = file_sub[index_slash+1:index_dot]

current_date = today = str (date.today())

extracted_file_name = file_sub_name+"Sentiment"+current_date+".txt"

file_extracted_handle = open(extracted_file_name,"a")

for i in range (0,len(key_phrases)):
  file_extracted_handle.write(key_phrases[i])
  


file_extracted_handle.close()